In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F  
import torchvision
import matplotlib.pyplot as plt
from torchvision import transforms
transforms = torchvision.transforms
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import gensim
from gensim import corpora
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras.layers import Embedding
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

nltk.download(['punkt','wordnet','stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# Device configuration

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# downloading dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IMDB Dataset.csv')

In [5]:
print(df.columns)
df.describe()

Index(['review', 'sentiment'], dtype='object')


,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [6]:
zero_1 = pd.get_dummies(df['sentiment'])
df = df.drop('sentiment', axis=1) 
df = df.join(zero_1)
df.head()

,review,negative,positive
0,One of the other reviewers has mentioned that ...,0,1
1,A wonderful little production. <br /><br />The...,0,1
2,I thought this was a wonderful way to spend ti...,0,1
3,Basically there's a family where a little boy ...,1,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",0,1


In [7]:
def text_processing(text):
  words=word_tokenize(text)

  words_without_punc=[]
  for w in words:
    if w.isalpha():
      words_without_punc.append(w.lower())

  clean_words=[]
  stop_words = set(stopwords.words("english"))
  for w in words_without_punc:
    if w not in stop_words:
      clean_words.append(w)
    
  lemmatized_words=[]
  for w in clean_words:
    lemmatized_words.append(WordNetLemmatizer().lemmatize(w))
  return ' '.join(lemmatized_words)

In [8]:
df['review'] = df['review'].apply(text_processing)
df.to_csv('/content/drive/MyDrive/NLPlay_with_Transformers/pre_processed_data.csv', index=False)
df.head()

,review,negative,positive
0,one reviewer mentioned watching oz episode hoo...,0,1
1,wonderful little production br br filming tech...,0,1
2,thought wonderful way spend time hot summer we...,0,1
3,basically family little boy jake think zombie ...,1,0
4,petter mattei love time money visually stunnin...,0,1


In [9]:
X = df.iloc[:, 0].values
Y = df.iloc[:, 2].values

In [10]:
max_words = 10000
max_len = 500
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
X = pad_sequences(sequences, maxlen=max_len)
X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle=True, random_state=60, test_size=0.20)
print(len(X_train))
print(len(X_test))

40000
10000


In [11]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model_rnn.h5',
    save_weights_only=False,
    mode='max',
    monitor='val_accuracy',
    save_best_only=True)

In [12]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 128))
model1.add(layers.SimpleRNN(10,dropout=0.5))
model1.add(layers.Dense(1,activation='sigmoid'))

model1.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
model1.fit(X_train, y_train, batch_size=256, epochs=10, validation_data=(X_test, y_test),verbose=1,callbacks = [model_checkpoint_callback])

Epoch 1/10
157/157 [==============================] - 76s 348ms/step - loss: 0.5782 - accuracy: 0.7123 - val_loss: 0.4133 - val_accuracy: 0.8352
Epoch 2/10
157/157 [==============================] - 55s 348ms/step - loss: 0.3685 - accuracy: 0.8597 - val_loss: 0.3826 - val_accuracy: 0.8448
Epoch 3/10
157/157 [==============================] - 54s 345ms/step - loss: 0.2962 - accuracy: 0.8876 - val_loss: 0.3460 - val_accuracy: 0.8642
Epoch 4/10
157/157 [==============================] - 54s 342ms/step - loss: 0.2386 - accuracy: 0.9143 - val_loss: 0.3389 - val_accuracy: 0.8728
Epoch 5/10
157/157 [==============================] - 54s 347ms/step - loss: 0.1949 - accuracy: 0.9312 - val_loss: 0.3444 - val_accuracy: 0.8682
Epoch 6/10
157/157 [==============================] - 56s 356ms/step - loss: 0.1671 - accuracy: 0.9417 - val_loss: 0.3555 - val_accuracy: 0.8704
Epoch 7/10
157/157 [==============================] - 56s 359ms/step - loss: 0.1483 - accuracy: 0.9501 - val_loss: 0.3636 - val_ac

In [13]:
model = tf.keras.models.load_model('model_rnn.h5')
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.8727999925613403
